In [1]:
# ! pip install python-dotenv
# ! pip install langchain --upgrade --no-cache-dir
! pip install wandb --upgrade --no-cache-dir


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%load_ext autoreload

In [18]:
%autoreload

import sys
import os
import json

os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "langchain-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" ) 
    
print( sys.path )

import util as du
path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import util_stopwatch as sw
import util_langchain as ulc
import genie_client as gc

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

from langchain import PromptTemplate
from langchain import LLMChain

import pandas as pd


from IPython.display import display, Markdown


[/var/genie-in-the-box/src/lib] already in sys.path
['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/var/genie-in-the-box/src/lib', '/var/genie-in-the-box/src']
Path [/var/genie-in-the-box/src] already in sys.path


In [19]:
pandas_prompt_template = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.

This is the result of `print(df.head())`:

"start_date","end_date","start_time","end_time","event_type","recurrent","recurrence_interval","priority_level","name","relationship","description_who_what_where"
"2023-07-01","2023-07-04","00:00","23:59","Concert",False,"","none","Jenny","coworker","Concert of Jenny at the city center"
"2023-07-01","2023-07-01","05:25","17:22","TODO",False,"","highest","Gregorio","friend","Send out invitations for the party for Gregorio"
"2023-07-01","2023-07-01","13:27","01:59","Appointment",False,"","high","Leroy Ruiz","father","Appointment with Leroy Ruiz at the clinic"
"2023-07-03","2023-07-04","00:00","23:59","Subscription",True,"1 year","highest","Leroy Ruiz","father","Renewal of Leroy Ruiz's subscription"
"2023-07-03","2023-07-04","00:00","23:59","Anniversary",True,"4 week","none","Juan","neighbor","Juan's anniversary celebration at the park"
"2023-07-04","2023-07-04","00:00","23:59","Anniversary",True,"4 week","medium","Leroy Ruiz","father","Leroy Ruiz's anniversary celebration at the park"

I want you to do three things for me:
1) Thoughts: Think out loud about what I'm asking you and how you're going to translate the question below into working python code that can be used to query `df` for the answer.
2) Code: A verbatim list of code, one line of code per item on the list
3) Explanation: Briefly and succinctly explain your code in plain English.

Format: return your response as a JSON object in the following fields:
{{
    "thoughts": "your thoughts",
    "code": [
        "count = df.shape[ 0 ]",
        "print( count )"
    ],
    "explanation": "your explanation"
}}

Begin!

Question: {question}
"""

In [27]:
langchain.debug = True
langchain.verbose = True

llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True, verbose=True )
raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

df_agent = create_pandas_dataframe_agent( llm, raw_df, verbose=True )

prompt = PromptTemplate(
    template=pandas_prompt_template,
    input_variables=["question"]
)
llm_chain = LLMChain(
   prompt=prompt,
   llm=llm
)
df_agent.memory = memory

In [33]:
def get_question_and_coda( event_query ):
    
    question_plus_coda_template = """
    {event_query}  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: The fields `start_date` and `end_date` are stored as strings. Make sure that you use the same data types when comparing or filtering dates.
    Hint: If your code returns any records, they should include all columns in the dataframe
    
    You must verify that your code runs to completion, once it does, return your response as a JSON string containing only these three fields: 
    1) answer: A simple, terse answer to the question '{event_query}'
    2) code: The Python code you generated in python list form, one line of code per item in the list, and 
    3) explanation: A brief explanation as to how the code works"""
    
    question = question_plus_coda_template.format( event_query=event_query )
    return question

question = get_question_and_coda( "What events do I have today?" )

for line in question.split( "\n" ): 
    print( line )



    What events do I have today?  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: The fields `start_date` and `end_date` are stored as strings. Make sure that you use the same data types when comparing or filtering dates.
    Hint: If your code returns any records, they should include all columns in the dataframe
    
    You must verify that your code runs to completion, once it does, return your response as a JSON string containing only these three fields: 
    1) answer: A simple, terse answer to the question 'What events do I have today?'
    2) code: The Python code you generated in python list form, one line of code per item in the list, and 
    3) explanation: A brief explanation as to how the code works


In [34]:
timer = sw.Stopwatch( "Running pandas prompt..." )
question = get_question_and_coda( "What events do I have today?" )
my_response = llm_chain.run( question=question )
timer.print( "Done!" )

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


Running pandas prompt...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "\n    What events do I have today?  \n    \n    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.\n    Hint: The fields `start_date` and `end_date` are stored as strings. Make sure that you use the same data types when comparing or filtering dates.\n    Hint: If your code returns any records, they should include all columns in the dataframe\n    \n    You must verify that your code runs to completion, once it does, return your response as a JSON string containing only these three fields: \n    1) answer: A simple, terse answer to the question 'What events do I have today?'\n    2) code: The Python code you generated in python list form, one line of code per item in the list, and \n    3) explanation: A brief explanation as to how the code works"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI]

In [35]:
response_dict = json.loads( my_response )
response_dict

{'thoughts': "First, I need to convert the 'start_date' and 'end_date' columns from string to datetime format. Then, I will filter the dataframe to only include rows where today's date falls within the start and end dates of the event. I will use the pandas 'to_datetime' function to convert the date columns and the 'datetime.now' function to get today's date. I will also use the 'between' function to filter the dataframe.",
 'code': ["df['start_date'] = pd.to_datetime(df['start_date'])",
  "df['end_date'] = pd.to_datetime(df['end_date'])",
  "today = pd.to_datetime('today')",
  "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
  'print(events_today)'],
 'explanation': "The code first converts the 'start_date' and 'end_date' columns to datetime format. Then it gets today's date. After that, it filters the dataframe to only include rows where today's date is between the start and end dates of the event. Finally, it prints the filtered dataframe, which includes

In [36]:
response_dict[ "code" ]

["df['start_date'] = pd.to_datetime(df['start_date'])",
 "df['end_date'] = pd.to_datetime(df['end_date'])",
 "today = pd.to_datetime('today')",
 "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
 'print(events_today)']

In [37]:
response = ulc.assemble_and_run_solution( response_dict[ "code" ], path=du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
for line in response[ "response" ].split( "\n" ):
    print( line )

Empty DataFrame
Columns: [start_date, end_date, start_time, end_time, event_type, recurrent, recurrence_interval, priority_level, name, relationship, description_who_what_where]
Index: []
